<a href="https://colab.research.google.com/github/Jayanths9/Dodelido_opencv/blob/main/Jay_Single_Image_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing Library Files
import albumentations as A
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import joblib
import json
from google.colab.patches import cv2_imshow

In [ ]:
dir_path='/content/drive/MyDrive/dodelido/'
image_dir="/content/drive/MyDrive/dodelido/dataset/"
data = pd.read_json(f"{dir_path}/labels.json").T
data = data.sort_index(ascending=True)
data = data.reset_index()

data["animal_color"] = data.apply(lambda row: [row[0], row[1]], axis=1)
data['labels'] = list(zip(data[0], data[1]))


In [ ]:
# Splitting the dataset

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    data['index'], data['animal_color'], test_size=0.2, random_state=44)

print("Number of posters for training: ", len(X_train))
print("Number of posters for validation: ", len(X_val))

Number of posters for training:  1712
Number of posters for validation:  428


In [ ]:
# The targets should be a list of list of strings to fit a binarizer (multi-hot encoding).

y_train = list(y_train)
y_val = list(y_val)

print(y_val[:4])


from sklearn.preprocessing import MultiLabelBinarizer

# Fit the multi-label binarizer on the training set
mlb = MultiLabelBinarizer()
mlb.fit(y_train)

# Loop over all labels and show them
N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

[['snake', 'blue'], ['lion', 'green'], ['lion', 'blue'], ['giraffe', 'green']]
0. alarm
1. blue
2. crane
3. giraffe
4. green
5. grey
6. lion
7. monkey
8. pink
9. sloth
10. snake
11. yellow


In [ ]:
# transform the targets of the training and test sets
y_train_bin = mlb.transform(y_train)
y_val_bin = mlb.transform(y_val)

# Print example of images and their binary targets
for i in range(3):
    print(X_train[i], y_train_bin[i])

0.png [1 0 0 0 0 0 0 0 0 0 0 0]
0_180.png [0 0 0 0 0 1 0 0 0 0 1 0]
0_270.png [0 1 0 0 0 0 0 0 0 0 1 0]


Workflow:

1.   Data collection
2.   Data preparation
3.   Create a fast input pipeline in TensorFlow
4.   Build up the model
5.   Get a transfer learning layer using TensorFlow Hub
6.   Stack a multi-label neural network classifier on top
7.   Model training and evaluation
8.   Understand the role of macro soft-F1 loss
9.   Export and save tf.keras models


In [ ]:
import tensorflow as tf

IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model
BATCH_SIZE = 256 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations

def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # filename=image_dir+filename
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label


def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    filenames=image_dir+filenames

    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)

    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)

    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)

    return dataset

In [ ]:
len(y_val_bin)

428

In [ ]:
train_ds1 = create_dataset(X_train, y_train_bin)
val_ds1 = create_dataset(X_val, y_val_bin)



Each batch will be a pair of arrays (one that holds the features and another one that holds the labels).
The features array will be of shape (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS).
The labels array will be of shape (BATCH_SIZE, N_LABELS) where N_LABELS is the maximum number of labels.



In [ ]:
import tensorflow_hub as hub
from keras import layers

feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))
feature_extractor_layer.trainable = False

In [ ]:
model = tf.keras.Sequential(
    [feature_extractor_layer,
    layers.Dense(1024, activation='relu', name='hidden_layer'),
    layers.Dense(N_LABELS, activation='sigmoid', name='output')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 hidden_layer (Dense)        (None, 1024)              1311744   
                                                                 
 output (Dense)              (None, 12)                12300     
                                                                 
Total params: 3582028 (13.66 MB)
Trainable params: 1324044 (5.05 MB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [ ]:
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.

    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)

    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost


@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)

    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive

    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
LR = 1e-4 # Keep it small when transfer learning
EPOCHS = 200
PATIENCE=5
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience = PATIENCE)
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=macro_soft_f1,
  metrics=[macro_f1])
# Train the model
history = model.fit(train_ds1,
                    epochs=EPOCHS,
                    validation_data=create_dataset(X_val, y_val_bin,
                                                   is_training=False),callbacks = [early_stop])

Epoch 1/200
7/7 [==============================] - 8s 808ms/step - loss: 0.1011 - macro_f1: 0.9836 - val_loss: 0.0989 - val_macro_f1: 0.9806
Epoch 2/200
7/7 [==============================] - 7s 1s/step - loss: 0.0832 - macro_f1: 0.9877 - val_loss: 0.0856 - val_macro_f1: 0.9795
Epoch 3/200
7/7 [==============================] - 5s 712ms/step - loss: 0.0698 - macro_f1: 0.9900 - val_loss: 0.0726 - val_macro_f1: 0.9825
Epoch 4/200
7/7 [==============================] - 5s 698ms/step - loss: 0.0594 - macro_f1: 0.9908 - val_loss: 0.0645 - val_macro_f1: 0.9834
Epoch 5/200
7/7 [==============================] - 5s 718ms/step - loss: 0.0509 - macro_f1: 0.9924 - val_loss: 0.0554 - val_macro_f1: 0.9868
Epoch 6/200
7/7 [==============================] - 5s 709ms/step - loss: 0.0434 - macro_f1: 0.9940 - val_loss: 0.0496 - val_macro_f1: 0.9879
Epoch 7/200
7/7 [==============================] - 8s 1s/step - loss: 0.0376 - macro_f1: 0.9950 - val_loss: 0.0441 - val_macro_f1: 0.9891
Epoch 8/200
7/7 [==

In [ ]:
X,y=val_ds1
images=X[0]
labels=np.array(X[1])

In [ ]:
labels

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [ ]:
predictions=np.array(model.predict(images))

8/8 [==============================] - 1s 48ms/step


In [ ]:
[predictions[0]]

[array([1.10601924e-07, 3.05853405e-06, 3.65717057e-08, 8.41973178e-06,
        7.31259206e-07, 3.00365082e-06, 5.42069188e-07, 3.62936028e-08,
        1.43271819e-07, 9.99999881e-01, 2.67302426e-07, 1.64907044e-06],
       dtype=float32)]

In [ ]:
# Define the threshold
threshold = 0.5

# Convert values to binary array based on the threshold
binary_array = np.where(np.array(predictions) > threshold, 1, 0)

In [ ]:
predicted_labels=mlb.inverse_transform(binary_array)
actual_labels=mlb.inverse_transform(labels)

In [ ]:
predicted_labels

[('sloth',),
 ('giraffe', 'green'),
 ('alarm',),
 ('crane', 'green'),
 ('pink', 'snake'),
 ('sloth',),
 ('crane', 'grey'),
 ('crane', 'yellow'),
 ('pink', 'snake'),
 ('crane', 'yellow'),
 ('alarm',),
 ('alarm',),
 ('sloth',),
 ('blue', 'lion'),
 ('giraffe', 'pink'),
 ('crane', 'yellow'),
 ('green', 'snake'),
 ('blue', 'giraffe'),
 ('crane', 'green'),
 ('blue', 'snake'),
 ('sloth',),
 ('blue', 'crane'),
 ('blue', 'lion'),
 ('monkey', 'pink'),
 ('blue', 'snake'),
 ('giraffe', 'pink'),
 ('sloth',),
 ('crane', 'pink'),
 ('blue', 'giraffe'),
 ('grey', 'monkey'),
 ('alarm',),
 ('pink', 'snake'),
 ('grey', 'snake'),
 ('crane', 'grey'),
 ('blue', 'snake'),
 ('crane', 'grey'),
 ('grey', 'monkey'),
 ('blue', 'monkey'),
 ('blue', 'monkey'),
 ('blue', 'giraffe'),
 ('green', 'lion'),
 ('grey', 'snake'),
 ('giraffe', 'pink'),
 ('giraffe', 'green'),
 ('alarm',),
 ('green', 'monkey'),
 ('sloth',),
 ('giraffe', 'pink'),
 ('giraffe', 'green'),
 ('giraffe', 'yellow'),
 ('grey', 'snake'),
 ('crane', 'pink

In [ ]:
for i in range(len(predicted_labels)):
  print(actual_labels[i],predicted_labels[i])

('sloth',) ('sloth',)
('giraffe', 'green') ('giraffe', 'green')
('alarm',) ('alarm',)
('crane', 'green') ('crane', 'green')
('pink', 'snake') ('pink', 'snake')
('sloth',) ('sloth',)
('grey', 'monkey') ('crane', 'grey')
('crane', 'yellow') ('crane', 'yellow')
('pink', 'snake') ('pink', 'snake')
('crane', 'yellow') ('crane', 'yellow')
('alarm',) ('alarm',)
('alarm',) ('alarm',)
('sloth',) ('sloth',)
('blue', 'lion') ('blue', 'lion')
('giraffe', 'pink') ('giraffe', 'pink')
('crane', 'yellow') ('crane', 'yellow')
('green', 'snake') ('green', 'snake')
('blue', 'giraffe') ('blue', 'giraffe')
('crane', 'green') ('crane', 'green')
('blue', 'snake') ('blue', 'snake')
('sloth',) ('sloth',)
('blue', 'crane') ('blue', 'crane')
('blue', 'lion') ('blue', 'lion')
('monkey', 'pink') ('monkey', 'pink')
('blue', 'snake') ('blue', 'snake')
('giraffe', 'pink') ('giraffe', 'pink')
('sloth',) ('sloth',)
('crane', 'pink') ('crane', 'pink')
('blue', 'giraffe') ('blue', 'giraffe')
('grey', 'monkey') ('grey', '

In [ ]:
tf.saved_model.save(model, f'{dir_path}/latest_model.h5')

In [ ]:
path="/content/drive/MyDrive/dodelido/three/IMG_1998.JPG"
cv2_image=cv2.imread(path)

In [165]:
def read_image(image_object):
  image_resized = tf.image.resize(image_object,[IMG_SIZE, IMG_SIZE])
  # Normalize it from [0, 255] to [0.0, 1.0]
  image_normalized = image_resized / 255.0
  expanded_tensor = tf.expand_dims(image_normalized, axis=0)
  ouptut=model.predict(expanded_tensor)
  result_list = list(ouptut)
  binary_array = np.where(np.array(result_list) > threshold, 1, 0)
  result=mlb.inverse_transform(binary_array)
  print(result)


def filter_overlapping_circles(circles, min_dist_between_circles):
  filtered_circles = []

  for circle in circles[0]:
      x, y, r = circle
      overlap = False
      for fc in filtered_circles:
          fx, fy, fr = fc
          distance = np.sqrt((x - fx) ** 2 + (y - fy) ** 2)
          if distance < min_dist_between_circles:
              overlap = True
              break
      if not overlap:
          filtered_circles.append(circle)

  return (filtered_circles)

In [187]:
path="/content/drive/MyDrive/dodelido/three/IMG_2017.JPG"
target_width = 1024
target_height = 1024
image=cv2.imread(path)
# image = cv2.resize(cv2_image, (target_width, target_height))


# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Use Hough Circle Transform to detect circles
blurred = cv2.GaussianBlur(gray, (11, 11), 0)
circles = cv2.HoughCircles(
    blurred,
    cv2.HOUGH_GRADIENT,
    dp=1,
    minDist=500,         # Adjust based on expected circle density
    param1=100,         # Higher threshold for Canny edge detector
    param2=30,          # Lower accumulator threshold for circle detection
    minRadius=100,       # Adjust based on the smallest expected circle size
    maxRadius=450       # Adjust based on the largest expected circle size
)

images=[]
buffer=20
if circles is not None:
    # Convert the (x, y) coordinates and radius of the circles to integers
    circles = circles.astype(int)
    min_dist_between_circles = 500  # Adjust this value as needed
    filtered_circles = filter_overlapping_circles(circles, min_dist_between_circles)

    # Loop over all detected circles
    for circle in filtered_circles:
        # Extract the coordinates and radius of the circle
        x, y, r = circle

        # Draw the bounding box around the circle
        x1 = max(0, x - r-buffer)
        y1 = max(0, y - r-buffer)
        x2 = min(image.shape[1], x + r+buffer)
        y2 = min(image.shape[0], y + r+buffer)

        # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Draw the bounding box on the original image
        new_image= image[y1:y2, x1:x2]
        images.append(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))



In [188]:
for i in images:
  read_image(i)

1/1 [==============================] - 0s 21ms/step
[('giraffe', 'pink')]
1/1 [==============================] - 0s 20ms/step
[('grey', 'snake')]
1/1 [==============================] - 0s 20ms/step
[('alarm',)]
